In [5]:
import tensorflow as tf
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2M
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
def preprocess_input(image):
    # Add custom preprocessing logic here if needed
    return image

def create_datagen():
    return ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input
    )

def create_generators(train_dir, validation_dir, test_dir, img_height, img_width, batch_size):
    train_datagen = create_datagen()
    validation_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)
    test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary'
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle=False,
        class_mode='binary'
    )

    return train_generator, validation_generator, test_generator

def build_model(img_height, img_width):
    base_model = EfficientNetV2M(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

class CustomModelCheckpoint(Callback):
    def __init__(self, save_path, monitor='val_accuracy', mode='max', verbose=1):
        super(CustomModelCheckpoint, self).__init__()
        self.save_path = save_path
        self.monitor = monitor
        self.mode = mode
        self.verbose = verbose
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if self.best_weights is None:
            self.best_weights = self.model.get_weights()
            self.best = current
        else:
            if (self.mode == 'max' and current > self.best) or (self.mode == 'min' and current < self.best):
                self.best = current
                self.best_weights = self.model.get_weights()
                if self.verbose > 0:
                    print(f'\nEpoch {epoch+1}: {self.monitor} improved to {self.best}, saving model to {self.save_path}')
                self.model.save(self.save_path)

    def on_train_end(self, logs=None):
        if self.best_weights is not None:
            self.model.set_weights(self.best_weights)

def create_callbacks(save_path):
    lr_scheduler = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-6,
        verbose=1
    )
    early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        verbose=1,
        mode='max'
    )
    checkpoint = CustomModelCheckpoint(
        save_path=save_path,
        monitor='val_accuracy',
        mode='max',
        verbose=1
    )
    return [lr_scheduler, early_stopping, checkpoint]

def train(train_dir, validation_dir, test_dir, img_height, img_width, batch_size, save_path):
    train_generator, validation_generator, test_generator = create_generators(
        train_dir, validation_dir, test_dir, img_height, img_width, batch_size)

    class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
    class_weights = dict(enumerate(class_weights))

    model = build_model(img_height, img_width)
    callbacks = create_callbacks(save_path)

    # Initial training with frozen layers
    history = model.fit(
        train_generator,
        epochs=5,
        validation_data=validation_generator,
        class_weight=class_weights,
        callbacks=callbacks
    )

    # Unfreeze the last block of layers and recompile
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False
        else:
            layer.trainable = True

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Continue training with unfrozen layers
    history_unfrozen = model.fit(
        train_generator,
        epochs=6,
        initial_epoch=4,
        validation_data=validation_generator,
        class_weight=class_weights,
        callbacks=callbacks
    )

    # Combine histories
    for key in history.history.keys():
        history.history[key].extend(history_unfrozen.history[key])

    return history

def predict_classes(test_generator, model):
    ypred = model.predict(test_generator)
    return ypred

def evaluate_model(ypred, y_true_classes, threshold=0.5):
    y_pred_classes = (ypred > threshold).astype(int)
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    cr = classification_report(y_true_classes, y_pred_classes, digits=4)
    print("Classification Report:\n", cr)

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, cmap='YlGnBu', fmt='d', xticklabels=['Normal', 'Porn'], yticklabels=['Normal', 'Porn'], annot_kws={'size': 16})
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j + 0.5, i + 0.2, '{:.2f}%'.format(cm_normalized[i, j] * 100), ha='center', va='bottom', color='black', fontsize=14)

    plt.ylabel('Actual', fontsize=12)
    plt.xlabel('Predicted', fontsize=12)
    plt.title('Confusion Matrix', fontsize=16)
    plt.show()

    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    print("Accuracy Score:", format(accuracy, '.4f'))

    f1 = f1_score(y_true_classes, y_pred_classes)
    print("F1 Score:", format(f1, '.4f'))

def tune_threshold(test_generator, model):
    ypred = model.predict(test_generator)
    thresholds = np.arange(0.0, 1.0, 0.01)
    best_f1 = 0
    best_threshold = 0

    for threshold in thresholds:
        y_pred_classes = (ypred > threshold).astype(int)
        f1 = f1_score(test_generator.classes, y_pred_classes)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    print(f"Best threshold: {best_threshold} with F1 score: {best_f1}")
    return best_threshold

In [8]:
train_dir = r'A:\AI DB\LSPD\Videos\photos_from_videos_pret\train'
validation_dir = r'A:\AI DB\LSPD\Videos\photos_from_videos_pret\val'
test_dir = r'A:\AI DB\LSPD\Videos\photos_from_videos_pret\test'
save_path = r'A:\AI DB\LSPD\models\efficient_trained_on_videos.keras'
img_height, img_width = 380, 380
batch_size = 64

In [11]:
# Training the model
print("Training starts:")
histories = train(train_dir, validation_dir, test_dir, img_height, img_width, batch_size, save_path)

print("Training finished.")
# Load the best model
model = tf.keras.models.load_model(save_path)

# Predict classes for evaluation
ypred = predict_classes(test_generator, model)

print("Evaluation:")
evaluate_model(ypred, test_generator.classes)

print("Calculating best threshold:")
best_threshold = tune_threshold(test_generator, model)

print("Re-evaluation with best threshold:")
evaluate_model(ypred, test_generator.classes, best_threshold)

Training starts:
Found 430877 images belonging to 2 classes.
Found 23937 images belonging to 2 classes.
Found 23939 images belonging to 2 classes.
Epoch 1/5
6733/6733 [==============================] - 11038s 2s/step - loss: 0.6794 - accuracy: 0.5651 - val_loss: 0.6631 - val_accuracy: 0.6084 - lr: 0.0010
Epoch 2/5
6733/6733 [==============================] - 10932s 2s/step - loss: 0.6733 - accuracy: 0.5804 - val_loss: 0.6813 - val_accuracy: 0.5451 - lr: 0.0010
Epoch 3/5
6733/6733 [==============================] - ETA: 0s - loss: 0.6697 - accuracy: 0.5886
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
6733/6733 [==============================] - 11080s 2s/step - loss: 0.6697 - accuracy: 0.5886 - val_loss: 0.6697 - val_accuracy: 0.5835 - lr: 0.0010
Epoch 4/5
6733/6733 [==============================] - ETA: 0s - loss: 0.6650 - accuracy: 0.5994
Epoch 4: val_accuracy improved to 0.6188745498657227, saving model to A:\AI DB\LSPD\models\efficient_trained_on_vide

ResourceExhaustedError: Graph execution error:

Detected at node 'model_1/block2b_expand_activation/Sigmoid' defined at (most recent call last):
    File "C:\ProgramData\anaconda3\envs\amchibrk\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\anaconda3\envs\amchibrk\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Mohamed ali\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\Mohamed ali\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\Mohamed ali\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\Mohamed ali\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 205, in start
  